In [387]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from mlens.visualization import corrmat
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline


In [2]:
from matplotlib import pyplot
from keras.applications import VGG19
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D,concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
train = pd.read_json("input/train.json")

In [4]:
def collect_test():
    test_data = pd.DataFrame()
    for i in range(4):
        test_piece = pd.read_csv('input/test_subparts/test'+str(i+1)+'.csv')
        test_data = pd.concat([test_data,test_piece])
        test_data = test_data.reset_index().drop(['index'],axis = 1)
    return test_data

In [5]:
test_data = collect_test()

In [4]:
def stats_and_fill_na(data,training = True):
    if training ==True:
        data['b1_median'] = [np.median(i) for i in data['band_1']]
        data['b2_median'] = [np.median(i) for i in data['band_2']]
        data['b1_mean'] = [np.mean(i) for i in data['band_1']]
        data['b2_mean'] = [np.mean(i) for i in data['band_2']]
        data['b1_max'] = [np.max(i) for i in data['band_1']]
        data['b2_max'] = [np.max(i) for i in data['band_2']]
        data['b1_min'] = [np.min(i) for i in data['band_1']]
        data['b2_min'] = [np.min(i) for i in data['band_2']]
        data['b1_std'] = [np.std(i) for i in data['band_1']]
        data['b2_std'] = [np.std(i) for i in data['band_2']]
        data['b1_var'] = [np.var(i) for i in data['band_1']]
        data['b2_var'] = [np.var(i) for i in data['band_2']]
        global stats
        stats = data.columns[-12:]
        X_train = data[data['inc_angle']!='na'][stats]
        y_train = data[data['inc_angle']!='na']['inc_angle']
        lr_model = LinearRegression().fit(X_train,y_train)
        data['inc_angle'][data['inc_angle']=='na'] = lr_model.predict(data[stats])
    else:
        only_band1 = ['band1_'+str(i) for i in range(75*75)]
        only_band2 = ['band2_'+str(i) for i in range(75*75)]
        both_bands = only_band1+only_band2
        data['b1_median'] = np.median(data[only_band1],axis = 1)
        data['b2_median'] = np.median(data[only_band2],axis = 1)
        data['b1_mean'] = np.mean(data[only_band1],axis = 1)
        data['b2_mean'] = np.mean(data[only_band2],axis = 1)
        data['b1_max'] = np.max(data[only_band1],axis = 1)
        data['b2_max'] = np.max(data[only_band2],axis = 1)
        data['b1_min'] = np.min(data[only_band1],axis = 1)
        data['b2_min'] = np.min(data[only_band2],axis = 1)
        data['b1_std'] = np.std(data[only_band1],axis = 1)
        data['b2_std'] = np.std(data[only_band2],axis = 1)
        data['b1_var'] = np.var(data[only_band1],axis = 1)
        data['b2_var'] = np.var(data[only_band2],axis = 1)
    return data

In [5]:
train = stats_and_fill_na(train)

C:\Users\yltbe\Anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
test_data = stats_and_fill_na(test_data,training = False)

In [6]:
def prepare_band12(data,training = True):
    if training==True:
        X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in data["band_1"]])
        X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in data["band_2"]])
        target_train = data['is_iceberg']
        X_other = X_other = data.drop(['band_1','band_2','id','is_iceberg'],axis = 1)
        return X_band_1,X_band_2,target_train,X_other
    else :
        only_band1 = ['band1_'+str(i) for i in range(75*75)]
        only_band2 = ['band2_'+str(i) for i in range(75*75)]
        both_bands = only_band1+only_band2
        X_band_1=np.array(data[only_band1]).reshape(8424,75,75)
        X_band_2=np.array(data[only_band2]).reshape(8424,75,75)
        X_other = data.iloc[:,-13:]
        return X_band_1,X_band_2,X_other

In [7]:
X_band_1,X_band_2,target,X_others = prepare_band12(train)

In [11]:
X_band_test_1,X_band_test_2,X_test_others = prepare_band12(test_data,training=False)

In [8]:
def prepare_RGB(band1,band2,method = 'third mean'):
    
    if method=='third mean':
        third = (band1+band2)/2
        image = np.concatenate([band1[:, :, :, np.newaxis],
                                band2[:, :, :, np.newaxis], 
                                third[:, :, :, np.newaxis]], axis=-1)
    elif method=='third max':
        third = np.max(np.concatenate([band1[:, :, :, np.newaxis], band2[:, :, :, np.newaxis]],axis = -1),axis = -1)
        image = np.concatenate([band1[:, :, :, np.newaxis],
                                band2[:, :, :, np.newaxis], 
                                third[:, :, :, np.newaxis]], axis=-1)
    return image

In [9]:
X = prepare_RGB(X_band_1,X_band_2,method = 'third mean')

In [14]:
X_test = prepare_RGB(X_band_test_1,X_band_test_2,method = 'third mean')

In [10]:
def feature_extract_vgg19(X_input):

    base_model = VGG19(weights='imagenet',include_top=False,input_shape=(75,75,3),classes=1)
    X = base_model.output
    X = GlobalMaxPooling2D()(X)
    model = Model(inputs=base_model.input, outputs=X)

    block4_pool_features = model.predict(X_input)
    return block4_pool_features

In [11]:
X = feature_extract_vgg19(X)

In [234]:
# scaler = MinMaxScaler().fit(X_train_others)
# X_train_others = scaler.transform(X_train_others)
# X_valid_others = scaler.transform(X_valid_others)

In [12]:
def concat_X_and_others():
    X_whole = pd.concat([X_others,pd.DataFrame(X,columns = ['feature'+str(i+1) for i in range(X.shape[1])])],axis = 1)
    return X_whole

In [13]:
X_whole = concat_X_and_others()

In [14]:
X_whole['inc_angle'] = pd.to_numeric(X_whole['inc_angle'])

In [ ]:

X_train_cv,X_valid,X_train_others,X_valid_others, y_train_cv, y_valid = train_test_split(X, X_others,target, random_state=2,stratify = target, train_size=0.80)

In [54]:
X_train_cv,X_valid, y_train_cv, y_valid = train_test_split(X_whole,target, random_state=2,stratify = target, train_size=0.80)

C:\Users\yltbe\Anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [92]:
del X_train_cv
del y_train_cv
del X_valid
del y_valid

### Machine learning models

#### Parameter tuning

In [169]:
def pipeline_with_scaler(model,scaler = 'minmaxscaler'):
    if scaler=='minmaxscaler':
        steps = [('minmaxscaler',MinMaxScaler()),('model',model)]
    elif scaler=='standardscaler':
        steps = [('standardscaler',StandardScaler()),('model',model)]
    pl = Pipeline(steps)
    return pl

In [183]:
def search_param(model,param_dict,method = 'random',scaler_mode = None,n_iter = 40,scoring = 'accuracy',cv = 5,n_jobs = 4):
    if method == 'random':
        if scaler_mode:
            pl = pipeline_with_scaler(model,scaler = scaler_mode)
            param_dict = {'model__'+key:value for key,value in param_dict.items()}
            rs = RandomizedSearchCV(pl,param_distributions=param_dict,n_iter = n_iter,scoring = scoring,cv = cv,verbose = 10,n_jobs = n_jobs).fit(X_whole,target)
        else:
            rs = RandomizedSearchCV(model,param_distributions=param_dict,n_iter = n_iter,scoring = scoring,cv = cv,verbose = 10,n_jobs = n_jobs).fit(X_whole,target)
        return rs
    elif method == 'grid':
        if scaler_mode:
            pl = pipeline_with_scaler(model,scaler = scaler_mode)
            param_dict = {'model__'+key:value for key,value in param_dict.items()}
            gs = GridSearchCV(pl,param_grid=param_dict,scoring = scoring,cv = cv,verbose = 10,n_jobs=n_jobs).fit(X_whole,target)
        else:
            gs = GridSearchCV(model,param_grid=param_dict,scoring = scoring,cv = cv,verbose = 10,n_jobs=n_jobs).fit(X_whole,target)
        return gs

In [180]:
def get_search_result(bpm_search):
    print('best_params:{}'.format(bpm_search.best_params_))
    print('best_score:{}'.format(bpm_search.best_score_))
    

#### Logistic Regression

In [ ]:
lr = LogisticRegression(C = 0.01,tol = 1e-05)
param_dict = {'penalty':['l1','l2'],
              'tol':[0.001,0.0001,0.00001],
              'C':[10,1,0.1,0.01]}
rs_lr = search_param(model = lr,method = 'grid',scaler_mode = 'standardscaler',param_dict=param_dict,n_iter = 20)

In [151]:
lr_bpm = {'C': 0.01, 'penalty': 'l2', 'tol': 0.0001}

{'C': 0.01, 'fit_intercept': True, 'penalty': 'l2', 'tol': 1e-05}

#### K-nn

In [ ]:
knn = KNeighborsClassifier()
param_dict = {'n_neighbors':[5,9,13,17,21],
              'weights':['uniform','distance'],
              'p':[1,2]}
knn_bpm_search = search_param(model = knn,method = 'grid',scaler_mode = 'standardscaler',param_dict=param_dict,n_iter = 20,n_jobs=4)

In [186]:
knn_bpm = {'n_neighbors': 9, 'p': 2, 'weights': 'uniform'}
knn_bsc = 0.8223192019950125

best_params:{'n_neighbors': 9, 'p': 2, 'weights': 'uniform'}
best_score:0.8223192019950125


#### SVM

In [ ]:
svm = SVC(C = 10,gamma = 0.001)
param_dict = {'C':[100,10,1],
              'kernel':['rbf'],
              'gamma':[0.0001,0.001,0.01]}
svm_bpm_search = search_param(model = svm,method = 'grid',scaler_mode = None,param_dict=param_dict,n_iter = 20,n_jobs=4)

In [198]:
svm_bpm = {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
svm_bsc = 0.8728179551122195

best_params:{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
best_score:0.8728179551122195


#### Naive bayes

In [ ]:
nb = BernoulliNB()
param_dict = {'alpha':[0.001,0.0001,0.01]+list(np.arange(0.1,1,0.1))}
nb_bpm_search = search_param(model = nb,method = 'grid',scaler_mode = None,param_dict=param_dict,n_iter = 20,n_jobs=4)

In [253]:
nb_bpm = {'alpha': 0.001}
nb_bsc = 0.7051122194513716

best_params:{'alpha': 0.001}
best_score:0.7051122194513716


#### Random forest

In [ ]:
rf = RandomForestClassifier(n_estimators = 120,criterion = 'entropy',max_depth = 12,max_features = 0.5)
param_dict = {'n_estimators':[120],
              'criterion':['entropy'],
              'max_depth':[12],
              'max_features':[0.5]}
rf_bpm_search = search_param(model = rf,method = 'grid',scaler_mode = 'minmaxscaler',param_dict=param_dict,n_iter = 40,n_jobs=4)

In [280]:
rf_bpm = {'criterion': 'entropy', 'max_depth': 12, 'max_features': 0.5, 'n_estimators': 120}
rf_bsc = 0.8416458852867831

best_params:{'criterion': 'entropy', 'max_depth': 12, 'max_features': 0.5, 'min_samples_leaf': 1, 'n_estimators': 120}
best_score:0.8347880299251871


In [283]:
get_search_result(rf_bpm_search)

best_params:{'model__criterion': 'entropy', 'model__max_depth': 12, 'model__max_features': 0.5, 'model__n_estimators': 120}
best_score:0.8366583541147132


#### MLP

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes = (256,256),solver = 'sgd',alpha = 1e-05,batch_size = 64,learning_rate = 'adaptive',max_iter = 150)
param_dict = {'hidden_layer_sizes':[(512,512),(256,256),(100,100)],
              'learning_rate_init':[0.1,0.01,0.001,0.0001],
              'learning_rate':['constant','adaptive'],
              'solver':['adam','sgd'],
              'alpha':[0.001,0.0001,0.00001],
              'batch_size':[32,64],
              'max_iter':[50,100,150,200],
              'early_stopping':[True,False]}
mlp_bpm_search = search_param(model = mlp,method = 'random',scaler_mode = None,param_dict=param_dict,n_iter = 20,n_jobs=4)

In [298]:
mlp_bpm = {'alpha': 1e-05, 'batch_size': 64, 'early_stopping': False, 'hidden_layer_sizes': (256, 256), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 150, 'solver': 'sgd'}
mlp_bsc = 0.8603491271820449

best_params:{'alpha': 1e-05, 'batch_size': 64, 'early_stopping': False, 'hidden_layer_sizes': (256, 256), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 150, 'solver': 'sgd'}
best_score:0.8603491271820449


#### GBM

In [ ]:
gbm = GradientBoostingClassifier(loss = 'exponential',n_estimators = 130,subsample = 0.8,max_depth = 5,max_features = 0.5)
param_dict = {'loss':['exponential'],
              'learning_rate':[0.1],
              'n_estimators':[120,130,140],
              'subsample':[0.7,0.8,0.9],
              'max_depth':[4,5,6],
              'max_features':[0.5,0.6,0.4]}
gbm_bpm_search = search_param(model = gbm,method = 'grid',scaler_mode = None,param_dict=param_dict,n_iter = 40,n_jobs=4)

In [307]:
gbm_bpm = {'learning_rate': 0.1, 'loss': 'exponential', 'max_depth': 5, 'max_features': 0.5, 'n_estimators': 130, 'subsample': 0.8}
gbm_bsc = 0.8603491271820449

best_params:{'subsample': 0.5, 'n_estimators': 150, 'max_features': 0.8, 'max_depth': 4, 'loss': 'exponential', 'learning_rate': 0.1}
best_score:0.8466334164588528


### Get models

In [332]:
def get_models():
    lr = LogisticRegression(C = 0.01,tol = 1e-05)
    knn = KNeighborsClassifier(n_neighbors = 9)
    svm = SVC(C = 10,gamma = 0.001,probability = True)
    nb = BernoulliNB(alpha = 0.001)
    rf = RandomForestClassifier(n_estimators = 120,criterion = 'entropy',max_depth = 12,max_features = 0.5)
    mlp = MLPClassifier(hidden_layer_sizes = (256,256),solver = 'sgd',alpha = 1e-05,
                        batch_size = 64,learning_rate = 'adaptive',max_iter = 150)
    gbm = GradientBoostingClassifier(loss = 'exponential',n_estimators = 130,
                                     subsample = 0.8,max_depth = 5,max_features = 0.5)
    models = dict(lr = lr,knn = knn,svm = svm,nb = nb,rf = rf,mlp = mlp,gbm = gbm)
    return models

In [333]:
models = get_models()

In [376]:
def evaluate_on_cv(model,X,y):
    single_model_pred_cv = pd.Series(np.zeros(shape = y.shape[0]))
    folds = list(StratifiedKFold(n_splits=5,shuffle = True,random_state = 7).split(X,y))
    for j,(train_mask,valid_mask) in enumerate(folds):
        X_train = X.iloc[train_mask]
        y_train = y.iloc[train_mask]
        X_valid = X.iloc[valid_mask]
        y_valid = y.iloc[valid_mask]
        fitted_model = model.fit(X_train,y_train)
        preds = fitted_model.predict_proba(X_valid)
        single_model_pred_cv.iloc[valid_mask] = preds[:,1]
        print('===========> '+'fold{} ,done!'.format(str(j+1)))
    acc = accuracy_score(y,single_model_pred_cv>0.5)
    return single_model_pred_cv,acc

In [377]:
def get_predict_matrix(model_dict):
    predict_proba_matrix = np.zeros(shape = (X_whole.shape[0],len(model_dict)))
    predict_proba_matrix = pd.DataFrame(predict_proba_matrix,columns = model_dict.keys())
    score_dict = dict()
    for i,(name,model) in enumerate(model_dict.items()):
        print('number '+str(i+1)+', '+name)
        single_pred_proba,single_acc = evaluate_on_cv(model = model,X = X_whole,y = target)
        predict_proba_matrix[name] = single_pred_proba
        score_dict[name] = single_acc
        print('number '+str(i+1)+', '+name+' finished!\n')
    return predict_proba_matrix,score_dict


In [378]:
predict_proba_matrix,score_dict = get_predict_matrix(get_models())

number 1, lr
===========> fold1 ,done!
===========> fold2 ,done!
===========> fold3 ,done!
===========> fold4 ,done!
===========> fold5 ,done!
number 1, lr finished!

number 2, knn
===========> fold1 ,done!
===========> fold2 ,done!
===========> fold3 ,done!
===========> fold4 ,done!
===========> fold5 ,done!
number 2, knn finished!

number 3, svm
===========> fold1 ,done!
===========> fold2 ,done!
===========> fold3 ,done!
===========> fold4 ,done!
===========> fold5 ,done!
number 3, svm finished!

number 4, nb
===========> fold1 ,done!
===========> fold2 ,done!
===========> fold3 ,done!
===========> fold4 ,done!
===========> fold5 ,done!
number 4, nb finished!

number 5, rf
===========> fold1 ,done!
===========> fold2 ,done!
===========> fold3 ,done!
===========> fold4 ,done!
===========> fold5 ,done!
number 5, rf finished!

number 6, mlp
===========> fold1 ,done!
===========> fold2 ,done!
===========> fold3 ,done!
===========> fold4 ,done!
===========> fold5 ,done!
number 6, mlp fin

In [384]:
from seaborn import diverging_palette

In [406]:
predict_proba_matrix['target'] = target



In [408]:
X_base,X_meta,y_base,y_meta = train_test_split(predict_proba_matrix[predict_proba_matrix.columns[:-1]],
                                               predict_proba_matrix['target'],test_size = 0.2,
                                               stratify = predict_proba_matrix['target'],random_state = 3)

In [420]:
dm_data = xgb.DMatrix(data = predict_proba_matrix[predict_proba_matrix.columns[:-1]],label = target)
params = {'objective':'binary:logistic','n_estimators':200,'colsample_bytree':0.8}
cv_results = xgb.cv(params = params,dtrain=dm_data,verbose_eval = True,num_boost_round=100,early_stopping_rounds=50,nfold = 5,stratified=True,metrics='logloss')

[0]	train-logloss:0.512753+0.0035705	test-logloss:0.535215+0.00603173
[1]	train-logloss:0.407731+0.00504009	test-logloss:0.448227+0.00950812
[2]	train-logloss:0.339729+0.00511722	test-logloss:0.396575+0.0113748
[3]	train-logloss:0.292959+0.00494696	test-logloss:0.360675+0.0136639
[4]	train-logloss:0.257447+0.00542595	test-logloss:0.338018+0.0163633
[5]	train-logloss:0.232045+0.00503845	test-logloss:0.323615+0.0188786
[6]	train-logloss:0.211402+0.00635307	test-logloss:0.314356+0.0200092
[7]	train-logloss:0.196832+0.00589052	test-logloss:0.308533+0.0216577
[8]	train-logloss:0.183173+0.00528433	test-logloss:0.305862+0.0233732
[9]	train-logloss:0.17178+0.00385619	test-logloss:0.304048+0.0256512
[10]	train-logloss:0.163757+0.0047561	test-logloss:0.303335+0.0268677
[11]	train-logloss:0.156889+0.00647785	test-logloss:0.303786+0.0275161
[12]	train-logloss:0.150807+0.00610447	test-logloss:0.303824+0.0281766
[13]	train-logloss:0.146395+0.00591307	test-logloss:0.303511+0.0307234
[14]	train-loglos

In [417]:
log_loss(target,predict_proba_matrix[predict_proba_matrix.columns[:-1]].mean(axis = 1))

0.32701484098543204

{'lr': 4}

In [169]:
def try_a_model(model,X_train,y_train,X_valid,y_valid):
    train = model.fit(X_train,y_train)
    train_acc = train.score(X_train,y_train)
    train_loss = log_loss(y_train,train.predict_proba(X_train))
    valid_acc = train.score(X_valid,y_valid)
    valid_loss = log_loss(y_valid,train.predict_proba(X_valid))
    print('Train Acc: {}'.format(train_acc))
    print('Valid Acc: {}'.format(valid_acc))
    print('Train Loss: {}'.format(train_loss))
    print('Valid Loss: {}'.format(valid_loss))
    return train_acc,valid_acc,train_loss,valid_loss

In [288]:
train_acc,valid_acc,train_loss,valid_loss = try_a_model(knn,X_train_cv,y_train_cv,X_valid,y_valid)

Train Acc: 0.8472330475448169
Valid Acc: 0.7850467289719626
Train Loss: 0.34870159122119554
Valid Loss: 0.4028870357934383


In [176]:
def cross_validation(model,X,y,cv = 5):
    cv_score_loss = -np.mean(cross_val_score(model,X,y,scoring='neg_log_loss',cv = cv))
    cv_score_acc = np.mean(cross_val_score(model,X,y,scoring='accuracy',cv = cv))
    print('Loss cv score: {}'.format(cv_score_loss))
    print('Acc cv score: {}'.format(cv_score_acc))
    return cv_score_loss,cv_score_acc

In [273]:
cv_score_loss,cv_score_acc = cross_validation(rf,X_whole,target,cv = 5)

Loss cv score: 0.46218617811203205
Acc cv score: 0.7917612009249047


In [154]:
def grid_search(model,X,y,param_grid,cv = 5):
    
    gdsearch1 = GridSearchCV(model,param_grid=param_grid,scoring = 'neg_log_loss',cv = cv)
    result1 = gdsearch1.fit(X,y)
    print('best param for loss: {}'.format(result1.best_params_))
    print('best score for loss: {}'.format(result1.best_score_))
    gdsearch2 = GridSearchCV(model,param_grid=param_grid,scoring = 'accuracy',cv = cv)
    result2 = gdsearch2.fit(X,y)
    print('best param for loss: {}'.format(result2.best_params_))
    print('best score for loss: {}'.format(result2.best_score_))
    return result1,result2

In [182]:
r1,r2 = grid_search(rf,X,target,param_grid = dict(n_estimators = list(range(10,60,10)),max_features = [128,256,512]))

best param for loss: {'max_features': 256, 'n_estimators': 40}
best score for loss: -0.4627297026361883
best param for loss: {'max_features': 128, 'n_estimators': 30}
best score for loss: 0.7992518703241895


### gridsearch with preprocessing

In [339]:

def pipelined_gridsearch(X,y,mlmodel,param_grid,scoring = 'accuracy',cv = 5):
    first = MinMaxScaler()
    normed_ml_search = Pipeline(steps=[('first',first),('mlmodel',mlmodel)])
    grid = GridSearchCV(normed_ml_search,param_grid=param_grid,scoring = scoring,cv = cv)
    grid.fit(X,y)
    best_param = grid.best_params_
    best_score = grid.best_score_
    print('Best param: {}'.format(best_param))
    print('Best score: {}'.format(best_score))
    return best_param,best_score

In [343]:
lr = LogisticRegression()
pipelined_gridsearch(X_whole,target,lr,param_grid={'mlmodel__C':[1+0.1*i for i in range(1,11)]},cv = 5)

Best param: {'mlmodel__C': 1.1}
Best score: 0.837281795511222


({'mlmodel__C': 1.1}, 0.83728179551122195)

In [381]:
knn = KNeighborsClassifier()
pipelined_gridsearch(X_whole,target,knn,param_grid={'mlmodel__n_neighbors':list(range(1,25,4))},cv = 7)

Best param: {'mlmodel__n_neighbors': 9}
Best score: 0.7743142144638404


({'mlmodel__n_neighbors': 9}, 0.77431421446384041)

In [376]:
def ml_kfolds_evaluate(mlmodel,X,y,K=5,normalization = True):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True).split(X,y))
    y_valid_pred_log = 0.0*y
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = np.array(X)[train_idx]
        y_train_cv = y[train_idx]
        X_valid = np.array(X)[valid_idx]
        y_valid= y[valid_idx]
        if normalization==True:
            first = MinMaxScaler()
            normed_ml_search = Pipeline(steps=[('first',first),('mlmodel',mlmodel)]).fit(X_train_cv,y_train_cv)
        else:
            normed_ml_search = mlmodel.fit(X_train_cv,y_train_cv)
        preds = normed_ml_search.predict(X_valid)
        y_valid_pred_log[valid_idx] = preds
    return y_valid_pred_log

In [379]:
lr = LogisticRegression(C=1)
preds = ml_kfolds_evaluate(lr,X_whole,target,K=5,normalization=True)
accuracy_score(target,preds)


===================FOLD= 1

===================FOLD= 2

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)



===================FOLD= 3


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)



===================FOLD= 4


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)



===================FOLD= 5


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


0.84413965087281795

### xgboost

In [56]:

xgbclf = xgb.XGBClassifier(n_estimators = 200,colsample_bytree=0.8)
xgbclf.fit(X_train_cv,y_train_cv)
preds = xgbclf.predict(X_valid)
xgbclf.score(X_train_cv,y_train_cv)
xgbclf.score(X_valid,y_valid)

0.84423676012461057

In [36]:
t1 = time.time()

In [37]:
t2 = time.time()
t2-t1

3.167266368865967

In [18]:
import time

In [ ]:
DM_train = xgb.DMatrix(X_train_cv,y_train_cv)
DM_valid = xgb.DMatrix(X_valid,y_valid)
t1 = time.time()
params = {'objective':'binary:logistic','alpha':1,'tree_method':'gpu_hist'}
xgbclf = xgb.train(params = params,dtrain = DM_train,num_boost_round=100,evals = [(DM_train,'train'),(DM_valid,'valid')],early_stopping_rounds=100)
t2 = time.time()
t2-t1
#xgbclf.predict(DM_valid)

In [105]:
dm_data = xgb.DMatrix(data = X_whole,label = target)
params = {'objective':'binary:logistic','n_estimators':200,'colsample_bytree':0.5}
cv_results = xgb.cv(params = params,dtrain=dm_data,verbose_eval = True,num_boost_round=100,early_stopping_rounds=5,nfold = 5,stratified=True,metrics='logloss')

[0]	train-logloss:0.550286+0.00206975	test-logloss:0.586198+0.00890863
[1]	train-logloss:0.470306+0.00958932	test-logloss:0.530621+0.0147537
[2]	train-logloss:0.40707+0.00296914	test-logloss:0.49179+0.0104228
[3]	train-logloss:0.351958+0.0070862	test-logloss:0.465135+0.00752298
[4]	train-logloss:0.312793+0.00864052	test-logloss:0.442968+0.0087691
[5]	train-logloss:0.279917+0.0076576	test-logloss:0.426425+0.00709965
[6]	train-logloss:0.254217+0.00845198	test-logloss:0.413488+0.00684292
[7]	train-logloss:0.232065+0.00947995	test-logloss:0.402179+0.00766479
[8]	train-logloss:0.212765+0.00987895	test-logloss:0.393354+0.00817723
[9]	train-logloss:0.19542+0.00722489	test-logloss:0.384648+0.00819415
[10]	train-logloss:0.176595+0.0048563	test-logloss:0.381758+0.00888178
[11]	train-logloss:0.162076+0.00497059	test-logloss:0.378541+0.00773131
[12]	train-logloss:0.149078+0.00638988	test-logloss:0.373771+0.0110737
[13]	train-logloss:0.135587+0.00562881	test-logloss:0.368797+0.0128146
[14]	train-lo

In [109]:
cv_results['test-logloss-mean'].iloc[-1]

0.34829900000000003

In [111]:
cv_results

,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.586198,0.008909,0.550286,0.002070
1,0.530621,0.014754,0.470306,0.009589
2,0.491790,0.010423,0.407070,0.002969
3,0.465135,0.007523,0.351958,0.007086
4,0.442968,0.008769,0.312793,0.008641
5,0.426425,0.007100,0.279917,0.007658
6,0.413488,0.006843,0.254217,0.008452
7,0.402179,0.007665,0.232065,0.009480
8,0.393354,0.008177,0.212765,0.009879
9,0.384648,0.008194,0.195420,0.007225


### xgboost grid search 1


In [75]:
dm_data = xgb.DMatrix(data = X_whole,label = target)
gbm = xgb.XGBClassifier()
gs_params1 = {'learning_rate':[0.25],'max_depth':[6],'n_estimators':[130],'subsample':[1],'colsample_bytree':[1],'gamma':[0,2,4,6,8,10]}
gs1 = GridSearchCV(gbm,param_grid=gs_params1,scoring = 'accuracy',cv = 5,verbose = 10,n_jobs=4)
gs1.fit(X_whole,target)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   37.9s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done  27 out of  30 | elapsed:  2.4min remaining:   15.9s
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  2.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'learning_rate': [0.25], 'max_depth': [6], 'n_estimators': [130], 'subsample': [1], 'colsample_bytree': [1], 'gamma': [0, 2, 4, 6, 8, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=10)

In [76]:
gs1.best_params_

{'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.25,
 'max_depth': 6,
 'n_estimators': 130,
 'subsample': 1}

In [77]:
gs1.cv_results_['mean_test_score'][np.argmax(gs1.cv_results_['mean_test_score'])]

0.8559850374064838

In [78]:
gs1.cv_results_['mean_train_score'][np.argmax(gs1.cv_results_['mean_test_score'])]

C:\Users\yltbe\Anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


1.0

### xgb Random Search 1

In [66]:
def random_search(num = 10):
    
    dm_data = xgb.DMatrix(data = X_whole,label = target)
    gbm2 = xgb.XGBClassifier()
    gs_params2 = {'learning_rate':[0.05,0.1,0.2],
                  'max_depth':[6,7],
                  'n_estimators':[130,150,170],
                  'subsample':[1,0.9,0.8],
                  'colsample_bytree':[0.9,0.8,0.7,0.6],
                  'gamma':[0,1,2],
                  'min_child_weight':[1,2,3]}
    gs2 = RandomizedSearchCV(gbm2,param_distributions=gs_params2,n_iter=40,scoring = 'accuracy',cv = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 2),verbose = 10,n_jobs=4)
    all_scores = []
    all_params = pd.DataFrame({'colsample_bytree': [],
                  'gamma': [],
                  'learning_rate': [],
                  'max_depth': [],
                  'min_child_weight': [],
                  'n_estimators': [],
                  'subsample': []})
    for i in range(num):
        gs2.fit(X_whole,target)
        best_param = gs2.best_params_
        best_score = gs2.best_score_
        all_scores.append(best_score)
        cur_best_param= pd.DataFrame({key:[value] for key,value in best_param.items()})
        all_params = pd.concat([all_params,cur_best_param],axis = 0)
        print('num_{},done!'.format(i+1))
    all_params['acc'] = all_scores
    return all_params
        

In [ ]:
all_params4 = random_search(num = 3)

In [ ]:
all_params3 = random_search(num = 5)

In [75]:
xgb_param = all_params4.iloc[1].to_dict()

In [76]:
xgb_param.pop('acc',None)

0.8572319201995012

In [ ]:
xgb_param = {'colsample_bytree': 0.9,
             'gamma': 0.0,
             'learning_rate': 0.1,
             'max_depth': 6,
             'min_child_weight': 2.0,
             'subsample': 1.0,'objective':'binary:logistic'}

In [116]:
X_tra,X_val,y_tra,y_val = train_test_split(X_whole,target)
dmtra = xgb.DMatrix(data = X_tra,label = y_tra)
dmval = xgb.DMatrix(data = X_val,label = y_val)
xgb_param = {'colsample_bytree': 0.8,
                     'gamma': 0,
                     'learning_rate': 0.1,
                     'max_depth': 3,
                     'min_child_weight': 2.0,
                     'subsample': 0.8,
                     'objective':'binary:logistic',
                     'booster':'gbtree','save_period':2,'model_dir':'xgbmodels'}
xgbmodel = xgb.train(params = xgb_param,
                  dtrain = dmtrain,
                  evals = [(dmtrain,'train'),(dmvalid,'valid')],
                  num_boost_round=125,
                  verbose_eval=True)

[0]	train-error:0.266563	valid-error:0.286604
[1]	train-error:0.254092	valid-error:0.267913
[2]	train-error:0.217459	valid-error:0.221184
[3]	train-error:0.199532	valid-error:0.211838
[4]	train-error:0.198753	valid-error:0.190031
[5]	train-error:0.190959	valid-error:0.186916
[6]	train-error:0.195635	valid-error:0.183801
[7]	train-error:0.195635	valid-error:0.193146
[8]	train-error:0.191738	valid-error:0.196262
[9]	train-error:0.173811	valid-error:0.174455
[10]	train-error:0.172253	valid-error:0.183801
[11]	train-error:0.166797	valid-error:0.174455
[12]	train-error:0.16212	valid-error:0.158879
[13]	train-error:0.151208	valid-error:0.17134
[14]	train-error:0.14887	valid-error:0.161994
[15]	train-error:0.146532	valid-error:0.168224
[16]	train-error:0.141076	valid-error:0.168224
[17]	train-error:0.139517	valid-error:0.158879
[18]	train-error:0.138737	valid-error:0.155763
[19]	train-error:0.141855	valid-error:0.152648
[20]	train-error:0.13562	valid-error:0.158879
[21]	train-error:0.140296	v

In [76]:
1-xgbcv['test-error-mean'].iloc[-1]


0.85785580000000006

In [112]:

xgbmodel.load_model('xgbmodels/1')

In [134]:
def xgb_cv5_ensemble():
    folds = list(StratifiedKFold(n_splits=5,shuffle=True,random_state=2).split(X_whole,target))
    all_data_valid_score = target*0
    a_test = target*0
    for i,(train_mask,valid_mask) in enumerate(folds):
        X_train = X_whole.iloc[train_mask]
        y_train = target.iloc[train_mask]
        X_valid = X_whole.iloc[valid_mask]
        y_valid = target.iloc[valid_mask]
        dmtrain = xgb.DMatrix(data = X_train,label = y_train)
        dmvalid = xgb.DMatrix(data = X_valid,label = y_valid)
        xgb_param = {'colsample_bytree': 0.8,
                     'gamma': 0,
                     'learning_rate': 0.1,
                     'max_depth': 3,
                     'min_child_weight': 2.0,
                     'subsample': 0.8,
                     'objective':'binary:logistic',
                     'booster':'gbtree','save_period':2,'model_dir':'xgbmodels'}
        xgbmodel = xgb.train(params = xgb_param,
                          dtrain = dmtrain,
                          evals = [(dmtrain,'train'),(dmvalid,'valid')],
                          num_boost_round=125,
                          verbose_eval=True)
        xgbmodel.save_model('xgbmodels/'+str(i+1))
        temp_valid = xgbmodel.predict(dmvalid)
        all_data_valid_score.iloc[valid_mask] = temp_valid
        test = xgb.Booster(model_file = 'xgbmodels/'+str(i+1))
        temp2 = test.predict(dmvalid)
        a_test.iloc[valid_mask] = temp2
        print('Fold'+str(i+1)+',Done!')
    all_cv_valid_score = accuracy_score(target,1*(all_data_valid_score>0.5))
    just_a_test = accuracy_score(target,1*(a_test>0.5))
    print('all_cv_valid_score')
    return all_cv_valid_score,just_a_test

In [135]:
cv5_score,jt = xgb_cv5_ensemble()

[0]	train-error:0.226989	valid-error:0.267081
[1]	train-error:0.229329	valid-error:0.267081
[2]	train-error:0.225429	valid-error:0.257764
[3]	train-error:0.180967	valid-error:0.226708
[4]	train-error:0.178627	valid-error:0.232919
[5]	train-error:0.179407	valid-error:0.242236
[6]	train-error:0.173167	valid-error:0.242236
[7]	train-error:0.170827	valid-error:0.23913
[8]	train-error:0.176287	valid-error:0.242236
[9]	train-error:0.160686	valid-error:0.23913
[10]	train-error:0.148986	valid-error:0.21118
[11]	train-error:0.148986	valid-error:0.214286
[12]	train-error:0.144306	valid-error:0.217391
[13]	train-error:0.139626	valid-error:0.220497
[14]	train-error:0.136505	valid-error:0.214286
[15]	train-error:0.134165	valid-error:0.217391
[16]	train-error:0.133385	valid-error:0.217391
[17]	train-error:0.131825	valid-error:0.226708
[18]	train-error:0.127925	valid-error:0.220497
[19]	train-error:0.124805	valid-error:0.214286
[20]	train-error:0.119345	valid-error:0.214286
[21]	train-error:0.116225	

[50]	train-error:0.080281	valid-error:0.155763
[51]	train-error:0.076383	valid-error:0.155763
[52]	train-error:0.073266	valid-error:0.152648
[53]	train-error:0.073266	valid-error:0.158879
[54]	train-error:0.074045	valid-error:0.155763
[55]	train-error:0.070928	valid-error:0.155763
[56]	train-error:0.070148	valid-error:0.161994
[57]	train-error:0.068589	valid-error:0.155763
[58]	train-error:0.06703	valid-error:0.158879
[59]	train-error:0.068589	valid-error:0.152648
[60]	train-error:0.06781	valid-error:0.168224
[61]	train-error:0.066251	valid-error:0.158879
[62]	train-error:0.065472	valid-error:0.168224
[63]	train-error:0.063133	valid-error:0.152648
[64]	train-error:0.061574	valid-error:0.152648
[65]	train-error:0.059236	valid-error:0.155763
[66]	train-error:0.056898	valid-error:0.146417
[67]	train-error:0.056118	valid-error:0.149533
[68]	train-error:0.055339	valid-error:0.149533
[69]	train-error:0.051442	valid-error:0.152648
[70]	train-error:0.056898	valid-error:0.143302
[71]	train-erro

[100]	train-error:0.032736	valid-error:0.152648
[101]	train-error:0.034295	valid-error:0.152648
[102]	train-error:0.031956	valid-error:0.155763
[103]	train-error:0.030398	valid-error:0.161994
[104]	train-error:0.031956	valid-error:0.168224
[105]	train-error:0.02728	valid-error:0.161994
[106]	train-error:0.024942	valid-error:0.158879
[107]	train-error:0.02728	valid-error:0.158879
[108]	train-error:0.025721	valid-error:0.158879
[109]	train-error:0.024942	valid-error:0.161994
[110]	train-error:0.025721	valid-error:0.158879
[111]	train-error:0.024162	valid-error:0.155763
[112]	train-error:0.0265	valid-error:0.161994
[113]	train-error:0.023383	valid-error:0.165109
[114]	train-error:0.022603	valid-error:0.165109
[115]	train-error:0.022603	valid-error:0.165109
[116]	train-error:0.021044	valid-error:0.161994
[117]	train-error:0.020265	valid-error:0.158879
[118]	train-error:0.020265	valid-error:0.158879
[119]	train-error:0.020265	valid-error:0.158879
[120]	train-error:0.019486	valid-error:0.161

[27]	train-error:0.103583	valid-error:0.24375
[28]	train-error:0.102804	valid-error:0.25
[29]	train-error:0.101246	valid-error:0.24375
[30]	train-error:0.096573	valid-error:0.2375
[31]	train-error:0.095016	valid-error:0.246875
[32]	train-error:0.093458	valid-error:0.221875
[33]	train-error:0.094237	valid-error:0.21875
[34]	train-error:0.093458	valid-error:0.215625
[35]	train-error:0.0919	valid-error:0.19375
[36]	train-error:0.0919	valid-error:0.2125
[37]	train-error:0.088785	valid-error:0.215625
[38]	train-error:0.088006	valid-error:0.2
[39]	train-error:0.084891	valid-error:0.1875
[40]	train-error:0.081776	valid-error:0.184375
[41]	train-error:0.080997	valid-error:0.1875
[42]	train-error:0.081776	valid-error:0.190625
[43]	train-error:0.07866	valid-error:0.19375
[44]	train-error:0.076324	valid-error:0.18125
[45]	train-error:0.077882	valid-error:0.178125
[46]	train-error:0.077103	valid-error:0.18125
[47]	train-error:0.077103	valid-error:0.171875
[48]	train-error:0.076324	valid-error:0.17

In [131]:
test = xgb.Booster(model_file = 'xgbmodels/1')

array([  9.30216193e-01,   1.18495345e-01,   9.33574736e-01,
         1.19841965e-02,   3.89860310e-02,   3.05081531e-03,
         8.43689561e-01,   2.02422030e-02,   9.72251713e-01,
         8.12476221e-03,   9.59637284e-01,   9.42460179e-01,
         5.10363579e-01,   9.57942784e-01,   9.61259305e-01,
         7.74938893e-03,   9.10757542e-01,   9.02308524e-01,
         9.58071887e-01,   4.71686721e-02,   9.70711827e-01,
         1.60330147e-01,   7.19150960e-01,   9.55066085e-01,
         2.88649917e-01,   3.55880171e-01,   8.69590044e-01,
         8.60727787e-01,   5.44600114e-02,   8.00786078e-01,
         2.81799883e-01,   3.84505749e-01,   4.48864279e-03,
         9.22111332e-01,   2.82749087e-02,   8.20679545e-01,
         8.00827861e-01,   1.75811059e-03,   2.88332105e-02,
         9.20137286e-01,   1.43489540e-01,   9.79234219e-01,
         9.70732033e-01,   5.23065403e-02,   2.53462702e-01,
         8.12394321e-01,   4.48300660e-01,   2.83244699e-01,
         2.71717072e-01,

In [ ]:
def xgb_cv5_predict():
    folds = list(StratifiedKFold(n_splits=5,shuffle=True,random_state=2).split(X_whole,target))
    all_data_valid_score = target*0
    a_test = target*0
    for i,(train_mask,valid_mask) in enumerate(folds):
        X_train = X_whole.iloc[train_mask]
        y_train = target.iloc[train_mask]
        X_valid = X_whole.iloc[valid_mask]
        y_valid = target.iloc[valid_mask]
        dmtrain = xgb.DMatrix(data = X_train,label = y_train)
        dmvalid = xgb.DMatrix(data = X_valid,label = y_valid)
        xgb_param = {'colsample_bytree': 0.8,
                     'gamma': 0,
                     'learning_rate': 0.1,
                     'max_depth': 3,
                     'min_child_weight': 2.0,
                     'subsample': 0.8,
                     'objective':'binary:logistic',
                     'booster':'gbtree','save_period':2,'model_dir':'xgbmodels'}
        xgbmodel = xgb.train(params = xgb_param,
                          dtrain = dmtrain,
                          evals = [(dmtrain,'train'),(dmvalid,'valid')],
                          num_boost_round=125,
                          verbose_eval=True)
        xgbmodel.save_model('xgbmodels/'+str(i+1))
        temp_valid = xgbmodel.predict(dmvalid)
        all_data_valid_score.iloc[valid_mask] = temp_valid
        test = xgb.Booster(model_file = 'xgbmodels/'+str(i+1))
        temp2 = test.predict(dmvalid)
        a_test.iloc[valid_mask] = temp2
        print('Fold'+str(i+1)+',Done!')
    all_cv_valid_score = accuracy_score(target,1*(all_data_valid_score>0.5))
    just_a_test = accuracy_score(target,1*(a_test>0.5))
    print('all_cv_valid_score')
    return all_cv_valid_score,just_a_test

In [143]:
from mlens.ensemble import SuperLearner

[MLENS] backend: threading


In [141]:
clone('a')

TypeError: Cannot clone object ''a'' (type <class 'str'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.

### Cross Validation Folds

In [15]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches

def data_augmentation_with_others(X_input,X_others,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    gen1 = data_aug.flow(X_input,Y_input,batch_size = batch_size,seed = 1990)
    gen2 = data_aug.flow(X_input,X_others,batch_size = batch_size,seed = 1990)
    while True:
            Xi1 = gen1.next()
            Xi2 = gen2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [Xi1[0], Xi2[1]], Xi1[1]

def data_no_augmentation_with_others(X_input,X_others,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = False,
                             vertical_flip = False)#,fill_mode = 'constant',cval = 0)
    gen1 = data_aug.flow(X_input,Y_input,batch_size = batch_size,seed = 1990)
    gen2 = data_aug.flow(X_input,X_others,batch_size = batch_size,seed = 1990)
    while True:
            Xi1 = gen1.next()
            Xi2 = gen2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [Xi1[0], Xi2[1]], Xi1[1]



In [16]:

def cnnmodel():
    X_input = Input(shape = (75,75,3))
    #First composite layer
    X = Conv2D(64,kernel_size = (5,5))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Second composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Third composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Forth composite layer
    X = Conv2D(64,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Flatten layer
    X = Flatten()(X)
    
    #First dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Second dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Decision layer
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs=X_input,outputs=X)

    return model

def cnnmodel_with_others():
    input_other = Input(shape=(13,), name="other")
    X_other = Dense(16,activation = 'relu')(input_other)
    X_other = Dense(16,activation = 'relu')(X_other)
    X_input = Input(shape = (75,75,3))
    #First composite layer
    X = Conv2D(64,kernel_size = (5,5))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Second composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Third composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Forth composite layer
    X = Conv2D(64,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Flatten layer
    X = Flatten()(X)
    X = concatenate([X,X_other])
    
    #First dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Second dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Decision layer
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs=[X_input,input_other],outputs=X)

    return model

def model_compile(model,lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None):
   
    if weights_path:
        model.load_weights(weights_path)
    else:
        pass
    if freezing_layers:
        for layer in model.layers[:freezing_layers]:
            layer.trainable = False
    else:
        pass
    optimizer = Adam(lr = lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
    #optimizer = SGD(lr, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7, monitor = 'val_loss', verbose = 1, save_best_only = True):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, monitor=monitor, verbose = verbose, save_best_only=True)
    return [es,msave]


In [17]:

def normal_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit(X_train_cv, y_train_cv,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result

def data_augment_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit_generator(data_augmentation(X_train_cv,y_train_cv,batch_size = batch_size),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result

def data_augment_fit_with_others(model,callbacks,epochs = 100,batch_size = 32,verbose = 1,da = False):
    
    if da == False:
        result = model.fit_generator(data_no_augmentation_with_others(X_train_cv,X_train_others,y_train_cv,batch_size = 32),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=([X_valid,X_valid_others], y_valid),
                          callbacks=callbacks)
    else:
        result = model.fit_generator(data_augmentation_with_others(X_train_cv,X_train_others,y_train_cv,batch_size = 32),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=([X_valid,X_valid_others], y_valid),
                          callbacks=callbacks)
    return result

In [21]:
Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
callbacks = get_callbacks(filepath = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/2018.1.4.hdf5', patience=30, save_best_only = True)
result = data_augment_fit_with_others(Cnnmodel,callbacks=callbacks,verbose = 2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 71, 71, 64)   4864        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 71, 71, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 35, 35, 64)   0           activation_1[0][0]               
__________________________________________________________________________________________________
dropout_1 

NameError: name 'X_train_cv' is not defined

In [ ]:
Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5')
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2_stage2.hdf5', patience=30, save_best_only = True)
result = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel.load_weights('model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage1.hdf5')
Cnnmodel.evaluate(X_valid, y_valid)
#Cnnmodel.evaluate(X_train_cv, y_train_cv)

In [ ]:
def save_history(result,name = 'fit_history.csv',path = os.getcwd()):
    train_loss = result.history['loss']
    val_loss = result.history['val_loss']
    train_accuracy = result.history['acc']
    val_accuracy = result.history['val_acc']
    fit_history = pd.DataFrame({'train_loss':train_loss,'val_loss':val_loss,'train_accuracy':train_accuracy,'val_accuracy':val_accuracy})
    fit_history.to_csv(os.path.join(path,name))

In [ ]:
save_history(result_Cnn)

### Prediction

In [37]:
Cnnmodel.predict([X_train_cv,X_train_others]).reshape(1283)+0

array([ 0.47316885,  0.43016976,  0.49740523, ...,  0.49190184,
        0.48540697,  0.48072121], dtype=float32)

In [47]:
def Model11_CV(K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X, target))
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        global X_train_cv
        X_train_cv = X[train_idx]
        global y_train_cv
        y_train_cv = target[train_idx]
        global X_valid
        X_valid = X[valid_idx]
        global y_valid
        y_valid= target[valid_idx]
        global X_train_others
        X_train_others = np.array(X_others)[train_idx]
        global X_valid_others
        X_valid_others = np.array(X_others)[valid_idx]
        
        scaler = MinMaxScaler().fit(X_train_others)
        X_train_others = scaler.transform(X_train_others)
        X_valid_others = scaler.transform(X_valid_others)
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
    
        file_path_stage1 = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv'+str(j+1)+'_stage1.hdf5'
        Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
        callbacks = get_callbacks(filepath = file_path_stage1, patience=30, save_best_only = True)
        result1 = data_augment_fit_with_others(Cnnmodel,callbacks = callbacks,epochs = 150,verbose=2)
        print('CV'+str(j+1)+', Done!')
        
        file_path_stage2 = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv'+str(j+1)+'_stage2.hdf5'
        Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = file_path_stage1)
        callbacks = get_callbacks(filepath = file_path_stage2, patience=30, save_best_only = True)
        result2 = data_augment_fit_with_others(Cnnmodel,callbacks = callbacks,epochs = 150,verbose=2,da=True)
        print('CV'+str(j+1)+', Done!')
#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        Cnnmodel.load_weights(filepath=file_path_stage2)
        #Getting Training Score
        score = Cnnmodel.evaluate([X_train_cv,X_train_others],y_train_cv)                                   
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate([X_valid,X_valid_others], y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])

        #Getting validation Score.
        pred_valid=Cnnmodel.predict([X_valid,X_valid_others])
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
#         temp_test=Cnnmodel.predict(X_test)
#         y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
#         temp_train=Cnnmodel.predict([X_train_cv,X_train_others])
#         y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

#     y_valid_pred_log=y_valid_pred_log/K
    #y_train_pred_log=y_train_pred_log/K

#     print('\n Train Log Loss Validation= ',log_loss(target, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target, y_valid_pred_log))
    return y_valid_pred_log

In [48]:
whole_valid = Model11_CV(K = 5)


===================FOLD= 1


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 71, 71, 64)   4864        input_16[0][0]                   
__________________________________________________________________________________________________
activation_61 (Activation)      (None, 71, 71, 64)   0           conv2d_61[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_61 (MaxPooling2D) (None, 35, 35, 64)   0           activation_61[0][0]              
__________________________________________________________________________________________________
dropout_91

Epoch 11/150
Epoch 00011: val_loss improved from 0.50535 to 0.45251, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.4855 - acc: 0.7514 - val_loss: 0.4525 - val_acc: 0.7733
Epoch 12/150
Epoch 00012: val_loss improved from 0.45251 to 0.43462, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.4472 - acc: 0.7712 - val_loss: 0.4346 - val_acc: 0.7857
Epoch 13/150
Epoch 00013: val_loss improved from 0.43462 to 0.40216, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.4229 - acc: 0.7895 - val_loss: 0.4022 - val_acc: 0.7981
Epoch 14/150
Epoch 00014: val_loss improved from 0.40216 to 0.37459, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.4001 - acc: 0.8154 - val_loss: 0.3746 - val_acc: 0.8

Epoch 56/150
Epoch 00056: val_loss improved from 0.26556 to 0.25648, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.2174 - acc: 0.9088 - val_loss: 0.2565 - val_acc: 0.8882
Epoch 57/150
Epoch 00057: val_loss did not improve
 - 6s - loss: 0.1864 - acc: 0.9237 - val_loss: 0.2641 - val_acc: 0.8851
Epoch 58/150
Epoch 00058: val_loss did not improve
 - 6s - loss: 0.1986 - acc: 0.9222 - val_loss: 0.2620 - val_acc: 0.8851
Epoch 59/150
Epoch 00059: val_loss did not improve
 - 5s - loss: 0.1822 - acc: 0.9222 - val_loss: 0.2701 - val_acc: 0.8851
Epoch 60/150
Epoch 00060: val_loss improved from 0.25648 to 0.25325, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv1_stage1.hdf5
 - 6s - loss: 0.1755 - acc: 0.9321 - val_loss: 0.2532 - val_acc: 0.8913
Epoch 61/150
Epoch 00061: val_loss did not improve
 - 6s - loss: 0.1822 - acc: 0.9306 - val_loss: 0.2627 - val_acc: 0.8913
Epoch 

...

Epoch 61/150
Epoch 00061: val_loss did not improve
 - 4s - loss: 0.1899 - acc: 0.9176 - val_loss: 0.2100 - val_acc: 0.8969
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 4s - loss: 0.1757 - acc: 0.9199 - val_loss: 0.2138 - val_acc: 0.9094
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1780 - acc: 0.9237 - val_loss: 0.2087 - val_acc: 0.9000
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.1788 - acc: 0.9268 - val_loss: 0.2130 - val_acc: 0.9094
Epoch 65/150
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.1958 - acc: 0.9199 - val_loss: 0.2025 - val_acc: 0.9094
Epoch 66/150
Epoch 00066: val_loss did not improve
 - 4s - loss: 0.1623 - acc: 0.9337 - val_loss: 0.2113 - val_acc: 0.9031
Epoch 67/150
Epoch 00067: val_loss did not improve
 - 4s - loss: 0.1597 - acc: 0.9367 - val_loss: 0.2208 - val_acc: 0.9094
Epoch 68/150
Epoch 00068: val_loss did not improve
 - 4s - loss: 0.1708 - acc: 0.9276 - val_loss: 0.2061 - val_acc: 0.9156
Epoch 69/150
Epo

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.68168, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage1.hdf5
 - 12s - loss: 1.3316 - acc: 0.5229 - val_loss: 0.6817 - val_acc: 0.5969
Epoch 2/150
Epoch 00002: val_loss improved from 0.68168 to 0.65134, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage1.hdf5
 - 5s - loss: 0.7251 - acc: 0.5334 - val_loss: 0.6513 - val_acc: 0.6031
Epoch 3/150
Epoch 00003: val_loss improved from 0.65134 to 0.62043, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage1.hdf5
 - 5s - loss: 0.6710 - acc: 0.5596 - val_loss: 0.6204 - val_acc: 0.6344
Epoch 4/150
Epoch 00004: val_loss improved from 0.62043 to 0.59054, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage1.hdf5
 - 5s - loss: 0.6343 - acc: 0.5905 - val_loss: 0.5905 - val_acc: 0.6469
Epo

Epoch 39/150
Epoch 00039: val_loss did not improve
 - 5s - loss: 0.2279 - acc: 0.9032 - val_loss: 0.2639 - val_acc: 0.8781
Epoch 40/150
Epoch 00040: val_loss did not improve
 - 5s - loss: 0.2249 - acc: 0.9077 - val_loss: 0.3077 - val_acc: 0.8500
Epoch 41/150
Epoch 00041: val_loss did not improve
 - 5s - loss: 0.2393 - acc: 0.8911 - val_loss: 0.3289 - val_acc: 0.8375
Epoch 42/150
Epoch 00042: val_loss did not improve
 - 5s - loss: 0.2298 - acc: 0.8955 - val_loss: 0.3282 - val_acc: 0.8438
Epoch 43/150
Epoch 00043: val_loss did not improve
 - 5s - loss: 0.2208 - acc: 0.9032 - val_loss: 0.3358 - val_acc: 0.8375
Epoch 44/150
Epoch 00044: val_loss did not improve
 - 5s - loss: 0.2533 - acc: 0.8917 - val_loss: 0.2891 - val_acc: 0.8656
Epoch 45/150
Epoch 00045: val_loss did not improve
 - 5s - loss: 0.2094 - acc: 0.9054 - val_loss: 0.2672 - val_acc: 0.8812
Epoch 46/150
Epoch 00046: val_loss improved from 0.25584 to 0.23998, saving model to model save/Model 11-Advanced CNN with DA, self TL and 

Epoch 1/150
Epoch 00001: val_loss improved from inf to 0.25216, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage2.hdf5
 - 10s - loss: 0.3359 - acc: 0.8545 - val_loss: 0.2522 - val_acc: 0.9031
Epoch 2/150
Epoch 00002: val_loss improved from 0.25216 to 0.24290, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage2.hdf5
 - 4s - loss: 0.3180 - acc: 0.8665 - val_loss: 0.2429 - val_acc: 0.9156
Epoch 3/150
Epoch 00003: val_loss did not improve
 - 4s - loss: 0.2871 - acc: 0.8673 - val_loss: 0.2568 - val_acc: 0.9062
Epoch 4/150
Epoch 00004: val_loss improved from 0.24290 to 0.22334, saving model to model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv5_stage2.hdf5
 - 4s - loss: 0.2641 - acc: 0.8818 - val_loss: 0.2233 - val_acc: 0.9156
Epoch 5/150
Epoch 00005: val_loss did not improve
 - 4s - loss: 0.2652 - acc: 0.8864 - val_loss: 0.2270 - val_acc: 0.9156
Epoch 6/1

Epoch 58/150
Epoch 00058: val_loss did not improve
 - 4s - loss: 0.1652 - acc: 0.9352 - val_loss: 0.2025 - val_acc: 0.9313
Epoch 59/150
Epoch 00059: val_loss did not improve
 - 4s - loss: 0.1878 - acc: 0.9215 - val_loss: 0.2115 - val_acc: 0.9281
Epoch 60/150
Epoch 00060: val_loss did not improve
 - 4s - loss: 0.1772 - acc: 0.9199 - val_loss: 0.2300 - val_acc: 0.9219
Epoch 61/150
Epoch 00061: val_loss did not improve
 - 4s - loss: 0.1826 - acc: 0.9216 - val_loss: 0.2142 - val_acc: 0.9281
Epoch 62/150
Epoch 00062: val_loss did not improve
 - 4s - loss: 0.1939 - acc: 0.9269 - val_loss: 0.2163 - val_acc: 0.9250
Epoch 63/150
Epoch 00063: val_loss did not improve
 - 4s - loss: 0.1773 - acc: 0.9283 - val_loss: 0.2286 - val_acc: 0.9187
Epoch 64/150
Epoch 00064: val_loss did not improve
 - 4s - loss: 0.1686 - acc: 0.9321 - val_loss: 0.2162 - val_acc: 0.9219
CV5, Done!
1284/1284 [==============================] - 2s 1ms/step
Train loss: 0.153598635135
Train accuracy: 0.939252336449
320/320 [====

### Submission

In [ ]:
def Evaluation_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X_train, target_train))
    y_test_pred_log=0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_valid = X_train[valid_idx]
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)


#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        #Getting Training Score
        print('cv'+str(j+1))
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])
        print('\n')

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [ ]:
preds = Evaluation_CV(X_train,K=5)
sub = test[['id']]
sub['is_iceberg'] = preds
sub.to_csv('5cv.csv',index = False)

### Submission(min max median stacking)

In [18]:
def test_scores(K=5):
    y_test_pred_log=0
    test_df = test_data[['id']]
    for j in range(K):
        print('\n===================FOLD=',j+1)
        file_path = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv'+str(j+1)+'.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = file_path)

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        test_df['cv'+str(j+1)]=temp_test.reshape(temp_test.shape[0])
    return test_df

def test_scores_scaled_others(X_test_others,K = 5):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X, target))
    test_df = test_data[['id']]
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        
        global X_train_others
        X_train_others = np.array(X_others)[train_idx]

        
        scaler = MinMaxScaler().fit(X_train_others)
        X_test_others = scaler.transform(X_test_others)

        file_path = 'model save/Model 11-Advanced CNN with DA, self TL and max layer(cv5)/Model-11 cv'+str(j+1)+'_stage2.hdf5'
        Cnnmodel = model_compile(cnnmodel_with_others(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = file_path)
        temp_test=Cnnmodel.predict([X_test,X_test_others])
        test_df['cv'+str(j+1)]=temp_test.reshape(temp_test.shape[0])
    return test_df


In [19]:
test_df = test_scores_scaled_others(X_test_others = X_test_others)


===================FOLD= 1


c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 71, 71, 64)   4864        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 71, 71, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 35, 35, 64)   0           activation_1[0][0]               
__________________________________________________________________________________________________
dropout_1 

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



===================FOLD= 2
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 71, 71, 64)   4864        input_2[0][0]                    
__________________________________________________________________________________________________
activation_5 (Activation)       (None, 71, 71, 64)   0           conv2d_5[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 35, 35, 64)   0           activation_5[0][0]               
_________________________________________________________________________________


===================FOLD= 4
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 71, 71, 64)   4864        input_4[0][0]                    
__________________________________________________________________________________________________
activation_13 (Activation)      (None, 71, 71, 64)   0           conv2d_13[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 35, 35, 64)   0           activation_13[0][0]              
_________________________________________________________________________________

In [20]:
test_df

,id,cv1,cv2,cv3,cv4,cv5
0,5941774d,5.667688e-02,5.530631e-02,8.635153e-03,1.127893e-03,4.170072e-03
1,4023181e,5.026714e-01,8.580565e-01,3.927149e-01,3.979664e-01,2.290074e-02
2,b20200e4,2.789924e-06,3.951226e-10,1.032073e-10,5.433600e-08,8.329140e-08
3,e7f018bb,9.994588e-01,9.999734e-01,9.998581e-01,9.999951e-01,9.999849e-01
4,4371c8c3,1.669144e-02,6.438503e-02,4.461252e-02,3.018984e-03,1.088171e-04
5,a8d9b1fd,1.406482e-01,1.800535e-02,2.911703e-03,8.031805e-03,2.462742e-03
6,29e7727e,1.855465e-01,1.123179e-01,7.316975e-02,5.049995e-03,2.020663e-02
7,92a51ffb,9.979691e-01,9.999943e-01,9.998738e-01,9.999973e-01,9.999726e-01
8,c769ac97,8.177361e-05,5.616964e-05,2.393075e-06,2.250952e-06,6.520145e-07
9,aee0547d,4.983280e-07,2.763404e-07,2.184263e-08,1.156752e-09,2.064336e-09


In [21]:
def get_sub_mmm(test_df):
    prediction = test_df
    prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.9,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.1,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),np.median(prediction[prediction.columns[1:]],axis = 1)))
    sub = prediction[['id','is_iceberg']]
    sub.to_csv('sub.csv',index = False)
    return sub

In [22]:
sub = get_sub_mmm(test_df)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Multiple submission stacking

In [28]:
path = 'prediction/stacking 1/'

In [34]:
sublist = os.listdir(path)
test_df = test_data[['id']]
for i in sublist:
    temp = pd.read_csv(os.path.join(path,i))
    test_df[i] = temp['is_iceberg']
test_df

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,id,lb0.1661.csv,lb0.1717.csv,lb0.1728.csv,lb0.1978.csv,lb0.2168.csv,mean-(2)(4)lb0.1717.csv
0,5941774d,4.883857e-02,3.457386e-02,7.377190e-03,3.609517e-03,2.721015e-03,8.815090e-02
1,4023181e,3.974485e-01,4.705284e-01,4.416436e-01,3.767341e-01,4.024012e-01,4.446322e-01
2,b20200e4,4.541493e-12,2.426515e-14,5.958568e-11,7.925581e-10,6.370236e-12,3.539300e-10
3,e7f018bb,9.999995e-01,9.999987e-01,9.999954e-01,9.999983e-01,9.999939e-01,9.999878e-01
4,4371c8c3,1.377228e-04,7.653040e-05,4.422425e-04,3.091512e-03,2.120113e-04,3.209088e-03
5,a8d9b1fd,4.990788e-02,8.502229e-02,5.834405e-02,6.308465e-02,7.421116e-04,2.790584e-02
6,29e7727e,1.211617e-01,1.527639e-01,1.079694e-01,1.907021e-01,1.918441e-02,1.444582e-02
7,92a51ffb,9.999950e-01,9.999959e-01,9.999992e-01,9.999897e-01,9.999989e-01,9.999560e-01
8,c769ac97,2.814740e-12,1.046281e-08,2.635662e-09,2.700044e-09,3.130832e-06,3.906417e-09
9,aee0547d,2.542710e-13,6.446605e-12,1.078889e-14,3.189674e-13,1.297212e-08,4.328927e-10


In [40]:
def get_sub_mmm(test_df):
    prediction = test_df
    prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.5,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.5,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),prediction['lb0.1661.csv']))
    sub = prediction[['id','is_iceberg']]
    sub.to_csv('sub.csv',index = False)
    return sub

In [41]:
sub = get_sub_mmm(test_df)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [39]:
sub

,id,is_iceberg
0,5941774d,2.721015e-03
1,4023181e,3.974485e-01
2,b20200e4,2.426515e-14
3,e7f018bb,9.999995e-01
4,4371c8c3,7.653040e-05
5,a8d9b1fd,7.421116e-04
6,29e7727e,1.444582e-02
7,92a51ffb,9.999992e-01
8,c769ac97,2.814740e-12
9,aee0547d,1.078889e-14
